In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
from statistics import mean

import requests

In [8]:
WR_data_complete = pd.read_csv('WR_data_complete.csv')

Here comes the fun part. 

Now have the complete data set of every NFL WR's statistics and Madden rating for the corresponding year. It is now time to classify these players so our KNN classifier has something to work with.

Let's take a look at Marvin Jones' stats and madden ratings. Marvin Jones is the only active receiver from the 2012 draft class, meaning he has recorded values for every season of our dataset (on second inspection, he missed the 2014 season due to injury).

What can be done to classify his career (thus far). He's had a long, fairly productive career at a glance. 10 seasons (11 actually but this data does not include 2023 season), one season over 1100 yards, a couple more close to 1000 (bench mark for a pro-bowl consideration usually) and a very high touchdown to reception ratio across his career. His stats are not Hall of Fame worthy by any stretch, but very solid. His Madden ratings suggest a good career as well.

Let's remember the goal. To predict ROI for the rookie class. Marvin Jones' best seasons were from 2016-2020 on Detriot, which is not the team that drafted him. If you were the general manager of the Cincinnati Bengals, Marvin Jones' success during those years is of little interest to you. So it makes sense that our model should only uses data in years where the player played for the team that drafted him.

Luckily for us, NFL rookie contracts are set at 4 years. After the 4 years, players become free agents and are free to sign wherever they please, sometimes resigning with the team that drafted them, or in Jones' case, opting to sign with another team. Thus, it makes sense to cap our model at the first four years of a players career to determine ROI.

Of course, it is possible that a player provides middling value for their franchise during these years, then resigns with the team, and sees a major uptick in production. For example, all-time great Antonio Brown (see stats below) spent his rookie contract (2010-2013) as a rotational player before having arguably the most productive 6-year stretch in NFL history after signing his second contract. However, examples like this are few and far between, as the NFL is a very fast-moving business; teams are always looking to improve talent so unproductive rookie contract players are usually not brought back. Thus, we can live with the occasional Antonio Brown-like career trajectory and the model should still be accurate. Julian Edelman also fits into this category.

[scroll]

In [14]:
WR_data_complete[WR_data_complete['name'] == 'Marvin Jones']

,id,name,position,team,season,season_type,receptions,targets,receiving_yards,receiving_tds,...,ypr,rec_td_percentage,rec_ypg,round,overall,ht,wt,forty,vertical,Overall Rating
1639,1209,Marvin Jones,WR,CIN,2012,REG,18,32,201,1,...,11.17,0.056,25.125000,5.0,166.0,2-Jun,199.0,4.46,33.0,70.0
1640,1209,Marvin Jones,WR,CIN,2013,REG,51,80,712,10,...,13.96,0.196,44.500000,5.0,166.0,2-Jun,199.0,4.46,33.0,81.0
1641,1209,Marvin Jones,WR,CIN,2015,REG,65,103,816,4,...,12.55,0.062,51.000000,5.0,166.0,2-Jun,199.0,4.46,33.0,82.0
1642,1209,Marvin Jones,WR,DET,2016,REG,55,103,930,4,...,16.91,0.073,62.000000,NaN,NaN,NaN,NaN,NaN,NaN,84.0
1643,1209,Marvin Jones,WR,DET,2017,REG,61,107,1101,9,...,18.05,0.148,68.812500,NaN,NaN,NaN,NaN,NaN,NaN,87.0
1644,1209,Marvin Jones,WR,DET,2018,REG,35,62,508,5,...,14.51,0.143,56.444444,NaN,NaN,NaN,NaN,NaN,NaN,86.0
1645,1209,Marvin Jones,WR,DET,2019,REG,62,91,779,9,...,12.56,0.145,59.923077,NaN,NaN,NaN,NaN,NaN,NaN,85.0
1646,1209,Marvin Jones,WR,DET,2020,REG,76,115,978,9,...,12.87,0.118,61.125000,NaN,NaN,NaN,NaN,NaN,NaN,83.0
1647,1209,Marvin Jones,WR,JAX,2021,REG,73,120,832,4,...,11.40,0.055,48.941176,NaN,NaN,NaN,NaN,NaN,NaN,82.0
1648,1209,Marvin Jones,WR,JAX,2022,REG,46,81,529,3,...,11.50,0.065,33.062500,NaN,NaN,NaN,NaN,NaN,NaN,78.0


In [27]:
WR_data_complete[WR_data_complete['name'] == 'Antonio Brown'].sort_values(by=['name','season'])

,id,name,position,team,season,season_type,receptions,targets,receiving_yards,receiving_tds,...,ypr,rec_td_percentage,rec_ypg,round,overall,ht,wt,forty,vertical,Overall Rating
162,84,Antonio Brown,WR,PIT,2012,REG,66,106,787,5,...,11.92,0.076,60.538462,6.0,195.0,10-May,186.0,4.56,33.5,86.0
163,84,Antonio Brown,WR,PIT,2013,REG,110,166,1499,8,...,13.63,0.073,93.687500,6.0,195.0,10-May,186.0,4.56,33.5,92.0
164,84,Antonio Brown,WR,PIT,2014,REG,129,181,1698,13,...,13.16,0.101,106.125000,6.0,195.0,10-May,186.0,4.56,33.5,97.0
165,84,Antonio Brown,WR,PIT,2015,REG,136,193,1834,10,...,13.49,0.074,114.625000,6.0,195.0,10-May,186.0,4.56,33.5,97.0
166,84,Antonio Brown,WR,PIT,2016,REG,106,154,1284,12,...,12.11,0.113,85.600000,6.0,195.0,10-May,186.0,4.56,33.5,97.0
167,84,Antonio Brown,WR,PIT,2017,REG,101,163,1533,9,...,15.18,0.089,109.500000,6.0,195.0,10-May,186.0,4.56,33.5,99.0
168,84,Antonio Brown,WR,PIT,2018,REG,104,168,1297,15,...,12.47,0.144,86.466667,6.0,195.0,10-May,186.0,4.56,33.5,98.0
161,84,Antonio Brown,WR,NE,2019,REG,4,8,56,1,...,14.00,0.250,56.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169,84,Antonio Brown,WR,TB,2020,REG,45,62,483,4,...,10.73,0.089,60.375000,NaN,NaN,NaN,NaN,NaN,NaN,86.0
170,84,Antonio Brown,WR,TB,2021,REG,42,62,545,4,...,12.98,0.095,77.857143,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Thus, Marvin Jones ROI for his rookie contract is given below.

In [28]:
WR_data_complete[WR_data_complete['name'] == 'Marvin Jones'].head(4)

,id,name,position,team,season,season_type,receptions,targets,receiving_yards,receiving_tds,...,ypr,rec_td_percentage,rec_ypg,round,overall,ht,wt,forty,vertical,Overall Rating
1639,1209,Marvin Jones,WR,CIN,2012,REG,18,32,201,1,...,11.17,0.056,25.125,5.0,166.0,2-Jun,199.0,4.46,33.0,70.0
1640,1209,Marvin Jones,WR,CIN,2013,REG,51,80,712,10,...,13.96,0.196,44.500,5.0,166.0,2-Jun,199.0,4.46,33.0,81.0
1641,1209,Marvin Jones,WR,CIN,2015,REG,65,103,816,4,...,12.55,0.062,51.000,5.0,166.0,2-Jun,199.0,4.46,33.0,82.0
1642,1209,Marvin Jones,WR,DET,2016,REG,55,103,930,4,...,16.91,0.073,62.000,NaN,NaN,NaN,NaN,NaN,NaN,84.0


Something else to think about, Marvin Jones was a fifth round pick. He was drafted 166 overall in the 2012 NFL draft as is shown in the data. A 900, 800, and 700 yard season in a four year span may not be ideal for a top 10 pick, but for a fifth round pick that is excellent production.

This introduces a new element: if I wish to include draft postion into my model (and I do), then my classifier has to represent relative success, not absolute. Furthermore, I would need to know where playes of the upcoming rookie class will be selected. The 2024 NFL draft is not until April 27, so I won't know this in time. Fortunately, there is usually a concensus for where prospects will land according to top scouts. E.g, top of the first round, second/third round, day 3 pick (fifth to seventh round), etc.